In [5]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk, ImageFilter, ImageEnhance


class PhotoEditor:
    def __init__(self, root):
        self.root = root
        self.root.title("Photo Editor")
        self.root.geometry("800x600")
        
        # Image variables
        self.original_image = None
        self.current_image = None
        self.display_image = None
        self.filename = None
        
        # Setup UI
        self.setup_ui()

    def setup_ui(self):
        """Create the user interface"""
        # Menu bar
        menubar = tk.Menu(self.root)
        
        # File menu
        file_menu = tk.Menu(menubar, tearoff=0)
        file_menu.add_command(label="Open", command=self.open_image)
        file_menu.add_command(label="Save", command=self.save_image)
        file_menu.add_separator()
        file_menu.add_command(label="Exit", command=self.root.quit)
        menubar.add_cascade(label="File", menu=file_menu)
        
        # Edit menu
        edit_menu = tk.Menu(menubar, tearoff=0)
        edit_menu.add_command(label="Reset", command=self.reset_image)
        menubar.add_cascade(label="Edit", menu=edit_menu)
        
        self.root.config(menu=menubar)
        
        # Image display
        self.image_frame = tk.Frame(self.root, bg="gray", bd=2, relief=tk.SUNKEN)
        self.image_frame.pack(pady=10, padx=10, fill=tk.BOTH, expand=True)
        
        self.image_label = tk.Label(self.image_frame, bg="gray")
        self.image_label.pack(fill=tk.BOTH, expand=True)
        
        # Control panel
        control_panel = tk.Frame(self.root, bg="#f0f0f0")
        control_panel.pack(fill=tk.X, padx=10, pady=5)
        
        # Filters section
        filter_frame = tk.LabelFrame(control_panel, text="Filters", bg="#f0f0f0")
        filter_frame.pack(side=tk.LEFT, padx=5)
        
        filters = [
            ("Blur", self.apply_blur),
            ("Sharpen", self.apply_sharpen),
            ("Grayscale", self.apply_grayscale),
            ("Emboss", self.apply_emboss),
            ("Edge Enhance", self.apply_edge_enhance)
        ]
        
        for text, command in filters:
            tk.Button(filter_frame, text=text, command=command, width=12).pack(side=tk.LEFT, padx=2, pady=2)
        
        # Adjustments section
        adjust_frame = tk.LabelFrame(control_panel, text="Adjustments", bg="#f0f0f0")
        adjust_frame.pack(side=tk.LEFT, padx=5)
        
        adjustments = [
            ("Brightness +", lambda: self.adjust_brightness(1.2)),
            ("Brightness -", lambda: self.adjust_brightness(0.8)),
            ("Contrast +", lambda: self.adjust_contrast(1.2)),
            ("Contrast -", lambda: self.adjust_contrast(0.8))
        ]
        
        for text, command in adjustments:
            tk.Button(adjust_frame, text=text, command=command, width=12).pack(side=tk.LEFT, padx=2, pady=2)
        
        # Action buttons
        action_frame = tk.Frame(control_panel, bg="#f0f0f0")
        action_frame.pack(side=tk.RIGHT, padx=5)
        
        tk.Button(action_frame, text="Reset", command=self.reset_image, width=10).pack(side=tk.LEFT, padx=2)
        tk.Button(action_frame, text="Save", command=self.save_image, width=10).pack(side=tk.LEFT, padx=2)

    def open_image(self):
        """Open an image file"""
        filetypes = [
            ("Image files", "*.jpg;*.jpeg;*.png;*.bmp;*.gif"),
            ("All files", "*.*")
        ]
        
        filename = filedialog.askopenfilename(title="Open Image", filetypes=filetypes)
        
        if filename:
            try:
                self.original_image = Image.open(filename)
                self.current_image = self.original_image.copy()
                self.filename = filename
                self.update_display()
            except Exception as e:
                messagebox.showerror("Error", f"Cannot open image: {str(e)}")

    def save_image(self):
        """Save the current image"""
        if not self.current_image:
            messagebox.showwarning("Warning", "No image to save!")
            return
            
        filetypes = [
            ("JPEG files", "*.jpg;*.jpeg"),
            ("PNG files", "*.png"),
            ("All files", "*.*")
        ]
        
        filename = filedialog.asksaveasfilename(
            defaultextension=".png",
            filetypes=filetypes,
            title="Save Image"
        )
        
        if filename:
            try:
                self.current_image.save(filename)
                messagebox.showinfo("Success", "Image saved successfully!")
            except Exception as e:
                messagebox.showerror("Error", f"Failed to save image: {str(e)}")

    def reset_image(self):
        """Reset to the original image"""
        if self.original_image:
            self.current_image = self.original_image.copy()
            self.update_display()
        else:
            messagebox.showwarning("Warning", "No image to reset!")

    def update_display(self):
        """Update the image display"""
        if self.current_image:
            # Get available display size
            max_width = self.image_frame.winfo_width() - 20
            max_height = self.image_frame.winfo_height() - 20
            
            # Calculate aspect ratio
            img_width, img_height = self.current_image.size
            ratio = min(max_width/img_width, max_height/img_height)
            new_size = (int(img_width * ratio), int(img_height * ratio))
            
            # Resize for display
            display_image = self.current_image.resize(new_size, Image.Resampling.LANCZOS)
            
            # Convert to PhotoImage
            self.display_image = ImageTk.PhotoImage(display_image)
            
            # Update the label
            self.image_label.config(image=self.display_image)
            self.image_label.image = self.display_image

    # Image processing functions
    def apply_blur(self):
        if self.current_image:
            self.current_image = self.current_image.filter(ImageFilter.BLUR)
            self.update_display()

    def apply_sharpen(self):
        if self.current_image:
            self.current_image = self.current_image.filter(ImageFilter.SHARPEN)
            self.update_display()

    def apply_grayscale(self):
        if self.current_image:
            self.current_image = self.current_image.convert("L").convert("RGB")
            self.update_display()

    def apply_emboss(self):
        if self.current_image:
            self.current_image = self.current_image.filter(ImageFilter.EMBOSS)
            self.update_display()

    def apply_edge_enhance(self):
        if self.current_image:
            self.current_image = self.current_image.filter(ImageFilter.EDGE_ENHANCE)
            self.update_display()

    def adjust_brightness(self, factor):
        if self.current_image:
            enhancer = ImageEnhance.Brightness(self.current_image)
            self.current_image = enhancer.enhance(factor)
            self.update_display()

    def adjust_contrast(self, factor):
        if self.current_image:
            enhancer = ImageEnhance.Contrast(self.current_image)
            self.current_image = enhancer.enhance(factor)
            self.update_display()

# Run the application
if __name__ == "__main__":
    root = tk.Tk()
    app = PhotoEditor(root)
    root.mainloop()